In [5]:
#imports
from LSTM import LSTM
import DataPreparationLSTM
import os
import pandas as pd

In [ ]:
input_data = pd.read_csv('data/SPY_data_5min.csv')
input_data['time'] = pd.to_datetime(input_data['time'])

# Filter trades within the time window from 09:30 to 16:00
input_data = input_data[(input_data['time'].dt.time >= pd.to_datetime('09:30').time()) & 
                 (input_data['time'].dt.time <= pd.to_datetime('16:00').time())]

input_data.drop(['Unnamed: 0','money','open','high','low'], axis=1, inplace=True)
input_data['time'] = pd.to_datetime(input_data['time']).dt.strftime('%H:%M')


In [2]:
learning_rates = [0.001, 0.01]
layer_one =[2,20,40]
layer_two = [2,15,40]
layer_three =[0,4,15]
layer_four = [0,2,10]

In [3]:
configs = {}

for i in range(len(learning_rates)):
    for j in range(len(layer_one)):
        for k in range(len(layer_two)):
            for c in range(len(layer_three)):
                for f in range(len(layer_four)):
                    configs["{}{}{}{}{}".format(i, j, k, c, f)] = [
                        learning_rates[i],
                        layer_one[j],
                        layer_two[k],
                        layer_three[c],
                        layer_four[f],
                    ]

In [3]:
futures = [1,5,20]

In [ ]:
models = {}

for future in futures:
    lstm = DataPreparationLSTM(
        df = input_data,
        future = future,
        lag = 20,
        min_max_scaler=True,
        log_transform=True,
        semi_variance=True,
        jump_detect=True,
        period_train= list(
            [
                pd.to_datetime("2004-01-02",format = "%Y-%m-%d"),
                pd.to_datetime("2010-12-31",format = "%Y-%m-%d"),
            ]
        ),
        period_validation = list(
            [
                pd.to_datetime("2004-01-02",format = "%Y-%m-%d"),
                pd.to_datetime("2010-12-31",format = "%Y-%m-%d"),  
            ]
        ),
        period_test= list(
            [
                pd.to_datetime("2011-01-01",format = "%Y-%m-%d"),
                pd.to_datetime("2013-12-31",format = "%Y-%m-%d"),
            ]
        ),
    )

    lstm.prepare_all()
    tf.keras.backend.clear_session()

In [ ]:
best_acc = 0
best_params = None

accs = []


for config in configs:
    model = LSTM(
        lstm.training_set,
        lstm.testing_set,
        lstm.validation_set,
        lstm.train_y,
        lstm.validation_set,
        lstm.test_y,
        epochs = 70, # mozda i ovo da bude prom
        learning_rate = config[0],
        layer_one = config[1],
        layer_two = config[2],
        layer_three = config[3],
        layer_four = config[4]
    )
    model.make_accurate_measures()

